In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 86.0/86.0 kB 805.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 977.5/977.5 kB 2.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125960 sha256=414140f44079f00fd7ac809ff7a650fbfb74725fcffa1e18da1e9fd3224e3fc9
  Stored in directory: C:\Users\user\AppData\Local\Temp\pip-ephem-wheel-cache-xwt60tsk\wheels\0a\f5\dd\9d00836c4e9e279c2a59d5b0ab72dafa66cbc626a327c550dd
Successfully built sentence_transformers


In [6]:
from sentence_transformers.datasets import ParallelSentencesDataset
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses, InputExample
from torch.utils.data import DataLoader
from sentence_transformers.datasets import ParallelSentencesDataset
from datetime import datetime
from scipy.spatial import distance


In [3]:
import torch
torch.cuda.is_available()

True

#Distillation training

In [2]:
max_seq_length = 256                #Student model max. lengths for inputs (number of word pieces)
train_batch_size = 64               #Batch size for training
inference_batch_size = 64           #Batch size at inference
max_sentences_per_language = 500000 #Maximum number of  parallel sentences for training
train_max_sentence_length = 250     #Maximum length (characters) for parallel training sentences

num_epochs = 5                       #Train for x epochs
num_warmup_steps = 10000             #Warumup steps

num_evaluation_steps = 1000          #Evaluate performance after every xxxx steps
dev_sentences = 1000

In [3]:



# model = SentenceTransformer('distilbert-base-nli-mean-tokens')
# teacher_model_name = 'paraphrase-distilroberta-base-v2'   #Our monolingual teacher model, we want to convert to multiple languages
# student_model_name = 'xlm-roberta-base'
# word_embedding_model = models.Transformer(student_model_name, max_seq_length=max_seq_length)
# # Apply mean pooling to get one fixed sized sentence vector
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# student_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# teacher_model = SentenceTransformer(teacher_model_name)
# train_data = ParallelSentencesDataset(student_model=student_model, teacher_model=teacher_model)
# train_data.load_data('/content/drive/MyDrive/RU-TT-2018-50000.txt')


# train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
# train_loss = losses.MSELoss(model=student_model)

In [4]:
# student_model.fit(train_objectives=[(train_dataloader, train_loss)],
#           #evaluator=evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: np.mean(scores)),
#           epochs=5,
#           warmup_steps=1000,
#           #evaluation_steps=num_evaluation_steps,
#           output_path='/content/drive/MyDrive/outp_st',
#           save_best_model=True,
#           optimizer_params= {'lr': 2e-5, 'eps': 1e-6}
#           )

In [ ]:
# emb = teacher_model.encode(['"О дивный новый мир" - изысканная и остроумная антиутопия о генетически программируемом обществе потребления, в котором разворачивается трагическая история Дикаря - "Гамлета" этого мира.Роман Олдоса Хаксли "О дивный новый мир" (1932) – одна из классических антиутопий. Разделенное на касты общество потребления не признает отклонения от выработанной нормы, поэтому никак не может вписаться в него тот, кто еще сохранил в себе остатки человечности – Дикарь'])
# emb1 = teacher_model.encode(['История семнадцатилетнего Себастьяна Барнака и трех людей, перевернувших его жизнь. Дядюшка Юстас, гедонист и бонвиван, научит юношу со вкусом наслаждаться земными радостями. Красивая и загадочная зрелая женщина Вероника Твейл станет его первой любовью. А другому дядюшке, мистику и искателю высшей истины Бруно, предстоит открыть Себастьяну, что в мире есть вещи куда важнее повседневной реальности…Каждый из этих героев несет свою идею и высказывает немало горьких и справедливых истин об устройстве человеческой души, общества и даже Вселенной.Сам Хаксли считал, что в этом романе ему удалось наиболее глубоко исследовать проблемы, встающие перед человеком XX века. Впервые на русском языке!'])

In [ ]:
# emb = teacher_model.encode(['Но Шельга с величайшим удивлением глядел уже не на мальчика, а мимо его плеча- на дверные стекла.'])
# emb1 = teacher_model.encode(['Ну'])

#Translation training

In [7]:
teacher_model = SentenceTransformer('all-mpnet-base-v2')

In [11]:
model = SentenceTransformer('paraphrase-distilroberta-base-v2')

In [18]:
import pandas as pd

excel_path = '/content/drive/MyDrive/RU-TT-2018-50000.xlsx'
df = pd.read_excel(excel_path)
df.columns = ['rus', 'tat']

In [19]:
train_samples = []

for i, row in enumerate(df.iloc[:25000, :].iterrows()):
  train_samples.append(InputExample(texts=[str(row[1]['rus']), str(row[1]['tat'])], label=1.0))

In [20]:
iter_lst = []
for row in df.iterrows():
  if type(row[1]['rus']) != int and type(row[1]['tat']) != int and type(row[1]['rus']) != float and type(row[1]['tat']) != float:
    iter_lst.append(row[1])

In [21]:
from random import randint
from tqdm import tqdm

for i, row in tqdm(enumerate(iter_lst[25000:])):
    for _ in range(2):
      idx = randint(0, len(iter_lst) - 1)
      if i != idx:
        texts = [iter_lst[idx]['tat'], row['tat']]
        # print(iter_lst[idx])
        try:
          emb = teacher_model.encode(str(iter_lst[idx]['rus']))
          emb1 = teacher_model.encode(str(row['rus']))
          label = 1 - distance.cosine(emb, emb1)
          # print(type(label))
          train_samples.append(InputExample(texts=texts, label=float(label)))
          # print(len(train_samples))
        except:
          pass

    for _ in range(1):
      idx = randint(0, len(iter_lst) - 1)
      if i != idx:
        texts = [iter_lst[idx]['tat'], row['rus']]
        try:
          emb = teacher_model.encode(str(iter_lst[idx]['rus']))
          emb1 = teacher_model.encode(str(row['rus']))
          label = 1 - distance.cosine(emb, emb1)
          train_samples.append(InputExample(texts=texts, label=float(label)))
          # train_samples.append(InputExample())
        except:
          pass

40it [00:00, 44.73it/s]


In [ ]:
import pickle
path_to_save = '/content/drive/MyDrive/dataset_1.pkl'
with open(path_to_save, 'wb') as file:
  pickle.dump(train_samples, file)
  file.close()



In [8]:
# import pickle
# with open('/content/drive/MyDrive/dataset_1.pkl', 'rb') as file:
#   train_samples = pickle.load(file)
#   file.close()

In [22]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)

In [ ]:
# from sklearn.model_selection import train_test_split

# train_data, val_data = train_test_split(train_samples)
# train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)
# # val_dataloader = DataLoader(val_data, shuffle=False, batch_size=16, collate_fn=collate_fn)

In [23]:
len(train_samples)

15030

In [24]:
# train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

# evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_data)

In [26]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          # evaluator=evaluator,
          epochs=10,
          # evaluation_steps=1000,
          warmup_steps=100,
          output_path='/content/drive/MyDrive/outp_model')

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/940 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [14]:
path_to_save_model = '/content/drive/MyDrive/outp_model'
model.save(path_to_save_model)

In [38]:
emb = model.encode([df.iloc[16545, :][1]])

In [39]:
emb1 = model.encode([df.iloc[16543, :][1]])

In [40]:
distance.cosine(emb[0], emb1[0])

0.18760192394256592